In [4]:
import matplotlib.pyplot as plt
import os
import numpy as np

In [5]:
dir_withDA = 'results/reacher-easy-10-14-s0-pixel-withDA'
dir_withoutDA = 'results/reacher-easy-10-15-s0-pixel-withoutDA'

In [6]:
withDA = np.load(os.path.join(dir_withDA, 'shiftability_data.npy'), allow_pickle=True)
withoutDA = np.load(os.path.join(dir_withoutDA, 'shiftability_data.npy'), allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'results/reacher-easy-10-14-s0-pixel-withDA/shiftability_data.npy'

In [ ]:
f, ax = plt.subplots(2, 1)
plt.suptitle('Reacher Easy @ 100K')